Hi all. In this notebook, I'll try to address the different image classification algorithms. I'll be working with a cola bottle dataset, which contains over 6500+ images segregated into 8 different classes. My approach is to make a comparitive study between custom CNN models and Transfer Learning models and check which would give the best accuracy.

If you like it, please upvote, and if you don't, please drop a comment, I would love to learn from my mistakes. Cheers! Let's get started! 

# Data Preprocessing

Before, we jump into the image classification algorithms, our preliminary step would be to preprocess the data. The data is in the form of images, we need to convert it to an array of vectors which would be given as input to the model for a prediction.

**Importing Libraries**

In [1]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
import pandas as pd
from IPython.display import Image
from keras.optimizers import Adam
from keras.layers import GlobalMaxPooling2D, Conv2D, MaxPooling2D, Flatten, Bidirectional, SpatialDropout2D, Input, Dropout
from sklearn.svm import SVC

**Preprocessing the data**

Basically, I'm creating a function where I'll be taking the input file as an argument, and I'd be converting the image to a numpy array using the inbuilt function **load_img** from **keras.preprocessing.image**

In [2]:
def prepare_image(file):
    #img_path = './practical example/'
    img = image.load_img(file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [3]:
X=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

X_train=X.flow_from_directory('../input/cola-bottle-identification/Soda Bottles',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=320,
                                                 class_mode='categorical',
                                                 shuffle=True)
label_map = (X_train.class_indices)
y=[]
for i in label_map:
    y.append(i)
y_categorical = pd.get_dummies(y)

Found 6615 images belonging to 8 classes.


In [4]:
CNN_Model1 = Sequential()
CNN_Model1.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224,224,3)))
CNN_Model1.add(MaxPooling2D(pool_size=(2, 2)))
CNN_Model1.add(Dropout(0.2))

CNN_Model1.add(Flatten())

CNN_Model1.add(Dense(128, activation='relu'))
CNN_Model1.add(Dense(8, activation='softmax'))
CNN_Model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 111, 111, 32)      0         
_________________________________________________________________
flatten (Flatten)            (None, 394272)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               50466944  
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 1032      
Total params: 50,468,872
Trainable params: 50,468,872
Non-trainable params: 0
____________________________________________

In [5]:
CNN_Model1.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
step_size_train=X_train.n//X_train.batch_size
CNN_Model1.fit(X_train,
                   steps_per_epoch=step_size_train,
                epochs=5)


Epoch 1/5
20/20 [==============================] - 172s 8s/step - loss: 19.0461 - accuracy: 0.1757
Epoch 2/5
20/20 [==============================] - 183s 9s/step - loss: 1.3059 - accuracy: 0.5248
Epoch 3/5
20/20 [==============================] - 169s 8s/step - loss: 0.7251 - accuracy: 0.8269
Epoch 4/5
20/20 [==============================] - 167s 8s/step - loss: 0.4501 - accuracy: 0.9295
Epoch 5/5
20/20 [==============================] - 168s 8s/step - loss: 0.2977 - accuracy: 0.9645


# Transfer Learning

> *Transfer learning (TL) is a research problem in machine learning (ML) that focuses on storing knowledge gained while solving one problem and applying it to a different but related problem. For example, knowledge gained while learning to recognize cars could apply when trying to recognize trucks.*


**MobileNet**

So, I'll be importing the MobileNet model with pretrained weights 'imagenet' and retrain the top layers with my input dataset. I'll keep the layers very limited and simple initially, and would add more layers if required.

I've used ImageDataGenerator function, whose task is to generate a dataset from image files. So, my X or dependent feature will be the preprocessed image dataset, and the target variable (y) will be the classes.

In [6]:
MobileNet_model = keras.applications.mobilenet.MobileNet()
MobileNet_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=MobileNet_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(8,activation='softmax')(x) #final layer with softmax activation

MobileNet_model=Model(inputs=MobileNet_model.input,outputs=preds)

for layer in MobileNet_model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in MobileNet_model.layers[:20]:
    layer.trainable=False
for layer in MobileNet_model.layers[20:]:
    layer.trainable=True


MobileNet_model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=X_train.n//X_train.batch_size
MobileNet_model.fit(X_train,
                   steps_per_epoch=step_size_train,
                epochs=5)




17227776/17225924 [==============================] - 1s 0us/step
Epoch 1/5
20/20 [==============================] - 592s 29s/step - loss: 1.6039 - accuracy: 0.4996
Epoch 2/5
20/20 [==============================] - 583s 29s/step - loss: 0.1158 - accuracy: 0.9524
Epoch 3/5
20/20 [==============================] - 593s 30s/step - loss: 0.0140 - accuracy: 0.9959
Epoch 4/5
20/20 [==============================] - 586s 29s/step - loss: 0.0304 - accuracy: 0.9924
Epoch 5/5
20/20 [==============================] - 578s 29s/step - loss: 0.0108 - accuracy: 0.9975
